In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import os
import pandas as pd
import numpy as np
import importlib.metadata
import seaborn as sns
import time
import scipy.stats as stats

from sklearn import metrics
from sklearn import model_selection

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


## Load and pre-process the data set

In [10]:
train = pd.read_csv('Data/train.csv')
print('loaded %d records' % len(train))

# Make sure all comment_text values are strings
train['comment_text'] = train['comment_text'].astype(str) 

# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# Convert taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

train = convert_dataframe_to_bool(train)

loaded 1804874 records


## Split the data into 80% train and 20% validate sets

In [11]:
train_df, validate_df = model_selection.train_test_split(train, test_size=0.2)
print('%d train comments, %d validate comments' % (len(train_df), len(validate_df)))


1443899 train comments, 360975 validate comments


## Create a text tokenizer

In [12]:
MAX_NUM_WORDS = 10000
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'

# Create a text tokenizer.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df[TEXT_COLUMN])

# All comments must be truncated or padded to be the same length.
MAX_SEQUENCE_LENGTH = 250
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

## Define and train a Convolutional Neural Net for classifying toxic comments

In [13]:
EMBEDDINGS_PATH = 'Embedding_file/glove.6B.100d.txt'
EMBEDDINGS_DIMENSION = 100
DROPOUT_RATE = 0.3
LEARNING_RATE = 0.00005
NUM_EPOCHS = 10
BATCH_SIZE = 128

def train_model(train_df, validate_df, tokenizer):
    # Prepare data
    train_text = pad_text(train_df[TEXT_COLUMN], tokenizer)
    train_labels = to_categorical(train_df[TOXICITY_COLUMN])
    validate_text = pad_text(validate_df[TEXT_COLUMN], tokenizer)
    validate_labels = to_categorical(validate_df[TOXICITY_COLUMN])

    # Load embeddings
    print('loading embeddings')
    embeddings_index = {}
    with open(EMBEDDINGS_PATH, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1,
                                 EMBEDDINGS_DIMENSION))
    num_words_in_embedding = 0
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            num_words_in_embedding += 1
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    # Create model layers.
    def get_convolutional_neural_net_layers():
        """Returns (input_layer, output_layer)"""
        sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
        embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDINGS_DIMENSION,
                            weights=[embedding_matrix],
                            trainable=False)

        x = embedding_layer(sequence_input)
        x = Conv1D(128, 2, activation='relu', padding='same')(x)
        x = MaxPooling1D(5, padding='same')(x)
        x = Conv1D(128, 3, activation='relu', padding='same')(x)
        x = MaxPooling1D(5, padding='same')(x)
        x = Conv1D(128, 4, activation='relu', padding='same')(x)
        x = MaxPooling1D(40, padding='same')(x)
        x = Flatten()(x)
        x = Dropout(DROPOUT_RATE)(x)
        x = Dense(128, activation='relu')(x)
        preds = Dense(2, activation='softmax')(x)
        return sequence_input, preds

    # Compile model.
    print('compiling model')
    input_layer, output_layer = get_convolutional_neural_net_layers()
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=LEARNING_RATE),
                  metrics=['acc'])

    # Train model.
    print('training model')
    model.fit(train_text,
              train_labels,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              validation_data=(validate_text, validate_labels),
              verbose=2)

    return model

model = train_model(train_df, validate_df, tokenizer)

loading embeddings
compiling model
training model
Epoch 1/10
11281/11281 - 332s - 29ms/step - acc: 0.9332 - loss: 0.1941 - val_acc: 0.9386 - val_loss: 0.1707
Epoch 2/10
11281/11281 - 326s - 29ms/step - acc: 0.9421 - loss: 0.1622 - val_acc: 0.9435 - val_loss: 0.1576
Epoch 3/10
11281/11281 - 326s - 29ms/step - acc: 0.9443 - loss: 0.1545 - val_acc: 0.9449 - val_loss: 0.1528
Epoch 4/10
11281/11281 - 326s - 29ms/step - acc: 0.9458 - loss: 0.1502 - val_acc: 0.9451 - val_loss: 0.1522
Epoch 5/10
11281/11281 - 325s - 29ms/step - acc: 0.9466 - loss: 0.1473 - val_acc: 0.9456 - val_loss: 0.1497
Epoch 6/10
11281/11281 - 324s - 29ms/step - acc: 0.9472 - loss: 0.1453 - val_acc: 0.9461 - val_loss: 0.1483
Epoch 7/10
11281/11281 - 322s - 29ms/step - acc: 0.9475 - loss: 0.1437 - val_acc: 0.9452 - val_loss: 0.1555
Epoch 8/10
11281/11281 - 310s - 28ms/step - acc: 0.9480 - loss: 0.1425 - val_acc: 0.9461 - val_loss: 0.1480
Epoch 9/10
11281/11281 - 316s - 28ms/step - acc: 0.9483 - loss: 0.1415 - val_acc: 0.94

## Generate model predictions on the validation set

In [14]:
MODEL_NAME = 'my_model'
validate_df[MODEL_NAME] = model.predict(pad_text(validate_df[TEXT_COLUMN], tokenizer))[:, 1]

11281/11281 ━━━━━━━━━━━━━━━━━━━━ 56s 5ms/step


In [15]:
validate_df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,my_model
720499,5003726,False,"Rep. Young, many islanders are low income too....",0.00,0.000000,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,1,0,0.0,0,4,0.002612
1709308,6217781,True,Jus take the damn phone.,0.04,0.746667,0.0,0.266667,0.0,NaN,NaN,...,approved,0,0,0,0,0,0.0,0,75,0.931845
579348,951090,False,It's akin to the opioid crisis that the G&M ha...,0.00,0.000000,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,1,2,0,0.0,0,4,0.006215
549291,915142,False,"And the Left learned nothing, thinking that sh...",0.00,0.000000,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,0,0,0.0,0,4,0.122622
58349,313387,False,I've been belittled by multiple jcpd officers....,0.00,0.000000,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,0,0,0.0,0,4,0.033353


## Define bias metrics, then evaluate our new model for bias using the validation set predictions

In [19]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = pd.concat([subgroup_negative_examples, non_subgroup_positive_examples])
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = pd.concat([subgroup_positive_examples, non_subgroup_negative_examples])
    return compute_auc(examples[label], examples[model_name])


def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {'subgroup': subgroup, 'subgroup_size': len(dataset[dataset[subgroup]]),
                  SUBGROUP_AUC: compute_subgroup_auc(dataset, subgroup, label_col, model),
                  BPSN_AUC: compute_bpsn_auc(dataset, subgroup, label_col, model),
                  BNSP_AUC: compute_bnsp_auc(dataset, subgroup, label_col, model)}
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

bias_metrics_df = compute_bias_metrics_for_model(validate_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df


,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
6,black,2956,0.787985,0.749373,0.958682
2,homosexual_gay_or_lesbian,2158,0.794058,0.768307,0.955694
7,white,5102,0.801040,0.764577,0.957968
5,muslim,4125,0.829505,0.800732,0.954900
4,jewish,1538,0.856885,0.852879,0.942829
8,psychiatric_or_mental_illness,958,0.870528,0.839838,0.955492
0,male,8876,0.878722,0.858674,0.946996
1,female,10696,0.879187,0.867375,0.943602
3,christian,8051,0.902929,0.908629,0.929281


## Calculate the final score

In [20]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    
get_final_metric(bias_metrics_df, calculate_overall_auc(validate_df, MODEL_NAME))

0.8833123736521704

## Prediction on Test data

In [21]:
test = pd.read_csv('Data/test.csv')
submission = pd.read_csv('Data/sample_submission.csv', index_col='id')

In [23]:
submission['prediction'] = model.predict(pad_text(test[TEXT_COLUMN], tokenizer))[:, 1]
submission.to_csv('benchmark_submission.csv')

3042/3042 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step
